## Optimizing NN architecture using Neuroevolution
Custom pipeline for model instantiation and neuroevolution procedure

### Must Remember points  
**__ getattribute __ can be used to convert string to a method or class call**

In [22]:
from sklearn import datasets
import torch
from torch import nn
from sklearn import model_selection
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt
import math

In [2]:
data = datasets.load_iris()
x = data['data']
y = data['target']

x_train, x_val, y_train, y_val = model_selection.train_test_split(data.data,data.target,test_size = 0.2)

x_tr_ten = torch.FloatTensor(x_train)
y_tr_ten = torch.LongTensor(y_train)

trainloader = TensorDataset(x_tr_ten,y_tr_ten)

x_te_ten = torch.FloatTensor(x_val)
y_te_ten = torch.LongTensor(y_val)

testloader = TensorDataset(x_te_ten,y_te_ten)

trainer = DataLoader(trainloader,batch_size=5)
tester = DataLoader(testloader,batch_size=5)

In [7]:
# #Experiments

# a = "sigmoid"
# b = torch.__getattribute__(a)

# t = b(torch.Tensor([1,0,9]))

In [19]:
class NN_Model(nn.Module):
    
    def __init__(self,layer_1,layer_2,activation_1,activation_2):
        
        super().__init__()
        
        self.hidden1 = nn.Linear(4,layer_1)
        
        self.hidden2 = nn.Linear(layer_1,layer_2)
        
        self.output = nn.Linear(layer_2,3)
        
        self.activation1 = torch.__getattribute__(activation_1)
        
        self.activation2 = torch.__getattribute__(activation_2)
        
    def forward(self,x):
        
        x = x.view(-1,4)
        
        x = self.activation1(self.hidden1(x))
        
        x = self.activation2(self.hidden2(x))
        
        x = torch.log_softmax(self.output(x),dim=1)
        
        return x

In [25]:
def convert_params(input_list):
    
    activation_fn = ["sigmoid","relu","tanh"]
    optimizer_fn = ["Adam","SGD","LBFGS"]
    
    layer1 = input_list[0]
    
    layer2 = input_list[1]
    
    activation1 = activation_fn[math.floor(input_list[2])]
    
    activation2 = activation_fn[math.floor(input_list[3])]
    
    learning_rate = input_list[4]
    
    optimizer = optimizer_fn[math.floor(input_list[5])]
    
    return layer1, layer2, activation1, activation2, learning_rate, optimizer    

In [43]:
def get_accuracy(params_list):
    
    """The function takes the input of the floating number encoded chromosome and returns the accuracy as thr output"""
    
    layer1, layer2, activation1, activation2, learning_rate, optimizer = convert_params(params_list)
    
    classifier = NN_Model(layer1, layer2, activation1, activation2)
    
    if torch.cuda.is_available:
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    classifier.to(device)

    loss_function = nn.NLLLoss()
    
    opt = torch.optim.__getattribute__(optimizer)
    
    optimizer = opt(classifier.parameters(),lr=learning_rate)
    
    train(classifier,optimizer,loss_function,trainer,tester)

In [52]:
get_accuracy([6,8,0.5,1.7,0.03,1.6])

77.7777777777778


In [49]:
def train(model,optimizer,loss_fun,trainloader,testloader,epochs=15,device="cuda"):
    
#     tr_list = []
#     val_list = []
    acc_list = []
#     epoch_list = [i+1 for i in range(epochs)]
    
    for epoch in range(epochs):
        
#         training_loss = 0
#         validation_loss = 0
        model.train()                #--------------------->Allows for parameters to be updated by backpropagation
        
        for batch in trainloader:
            
            optimizer.zero_grad()
            inputs,labels = batch
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = loss_fun(outputs,labels)
            loss.backward()
            optimizer.step()
            
#             training_loss += loss.item()
        
        model.eval()              #------------------------>Freezes the parameters for model validation
        correct_pred = 0
        total_pred = 0
        
        for batch in testloader:
            
            inputs,labels = batch
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = loss_fun(outputs,labels)
            
#             validation_loss +=loss.item()
            
            ps = torch.exp(outputs)         #-------------->The final activation function is Log_Softmax that's why we take exp
            
            correct = torch.eq(torch.max(ps,dim=1)[1],labels).view(-1)
            
            correct_pred += torch.sum(correct).item()
            total_pred += correct.shape[0]       #--------->Alternatively can also write batch.shape[0]
            
#         training_loss = training_loss/len(trainloader)
#         validation_loss = validation_loss/len(testloader)
        
#         tr_list.append(training_loss)
#         val_list.append(validation_loss)
        acc_list.append((correct_pred*100.0/total_pred))
    
    print(sum(acc_list)/len(acc_list))
            
#         print("Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}"
#               .format(epoch+1, training_loss,validation_loss, correct_pred * 100.0 / total_pred))
    
#     fig = plt.figure(figsize=(20,6))
        
#     plt.subplot(1,3,1)
#     plt.plot(epoch_list,tr_list)
#     plt.title("Training Loss")
    
#     plt.subplot(1,3,2)
#     plt.plot(epoch_list,val_list)
#     plt.title("Validation Loss")
    
#     plt.subplot(1,3,3)
#     plt.plot(epoch_list,acc_list)
#     plt.title("Accuracy")
    
#     plt.show()